In [18]:
import requests
from datetime import datetime

In [19]:
CIK = "0000002488"  # AMD
URL = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{CIK}.json"

headers = {
    "User-Agent": "MarketIntelligenceProject/1.0 your_email@example.com"
}

response = requests.get(URL, headers=headers)
data = response.json()


In [20]:
def is_full_year(record):
    start = datetime.fromisoformat(record["start"])
    end = datetime.fromisoformat(record["end"])
    return (end - start).days > 300


In [21]:
def pick_latest(records, fy_key="fy"):
    latest = {}

    for r in records:
        fy = r[fy_key]
        filed = datetime.fromisoformat(r["filed"])

        if fy not in latest or filed > latest[fy]["filed"]:
            latest[fy] = {
                "value": r["val"],
                "filed": filed
            }

    return {fy: v["value"] for fy, v in latest.items()}



def extract_flow_metric(metric_name, data, start_fy=2015, end_fy=2024):
    records = data["facts"]["us-gaap"].get(metric_name, {}).get("units", {}).get("USD", [])

    filtered = []

    for r in records:
        if r.get("form") != "10-K":
            continue
        if "start" not in r or "end" not in r:
            continue

        duration = (datetime.fromisoformat(r["end"]) -
                    datetime.fromisoformat(r["start"])).days
        if duration <= 300:
            continue

        derived_fy = datetime.fromisoformat(r["end"]).year
        if start_fy <= derived_fy <= end_fy:
            r["derived_fy"] = derived_fy
            filtered.append(r)

    return pick_latest(filtered, fy_key="derived_fy")

def extract_revenue(data, start_fy=2015, end_fy=2024):
    revenue_tags = [
        "Revenues",
        "RevenueFromContractWithCustomerIncludingAssessedTax",
        "RevenueFromContractWithCustomerExcludingAssessedTax",
        "SalesRevenueNet"
    ]

    combined = {}

    for tag in revenue_tags:
        records = data["facts"]["us-gaap"].get(tag, {}).get("units", {}).get("USD", [])
        filtered = []

        for r in records:
            if r.get("form") != "10-K":
                continue
            if "start" not in r or "end" not in r:
                continue

            duration = (datetime.fromisoformat(r["end"]) -
                        datetime.fromisoformat(r["start"])).days
            if duration <= 300:
                continue

            derived_fy = datetime.fromisoformat(r["end"]).year
            if start_fy <= derived_fy <= end_fy:
                r["derived_fy"] = derived_fy
                filtered.append(r)

        yearly = pick_latest(filtered, fy_key="derived_fy")

        for fy, val in yearly.items():
            if fy not in combined:
                combined[fy] = val

    return combined


def extract_stock_metric(metric_name, data, start_fy=2015, end_fy=2024):
    records = data["facts"]["us-gaap"].get(metric_name, {}).get("units", {}).get("USD", [])

    by_year = {}

    for r in records:
        if r.get("form") != "10-K":
            continue

        fy = r.get("fy")
        if fy is None or not (start_fy <= fy <= end_fy):
            continue

        filed = datetime.fromisoformat(r["filed"])
        by_year.setdefault(fy, []).append((filed, r["val"]))

    result = {}

    for fy, items in by_year.items():
        items.sort(reverse=True)  # latest filing first
        for _, val in items:
            if val is not None:
                result[fy] = val
                break

    return result

revenues = extract_revenue(data)
net_income = extract_flow_metric("NetIncomeLoss", data)
operating_cash_flow = extract_flow_metric(
    "NetCashProvidedByUsedInOperatingActivities", data
)

total_assets = extract_stock_metric("Assets", data)
raw_liabilities = extract_stock_metric("Liabilities", data)
equity = extract_stock_metric("StockholdersEquity", data)

total_liabilities = {}

for fy in range(2015, 2025):
    if fy in raw_liabilities:
        total_liabilities[fy] = raw_liabilities[fy]
    elif fy in total_assets and fy in equity:
        total_liabilities[fy] = total_assets[fy] - equity[fy]


In [22]:
final_data = []

for fy in range(2015, 2025):
    final_data.append({
        "company_name": "AMD",
        "fiscal_year": fy,
        "revenue": revenues.get(fy),
        "net_income": net_income.get(fy),
        "operating_cash_flow": operating_cash_flow.get(fy),
        "total_assets": total_assets.get(fy),
        "total_liabilities": total_liabilities.get(fy)
    })



In [23]:
for row in final_data:
    print(row)


{'company_name': 'AMD', 'fiscal_year': 2015, 'revenue': 3991000000, 'net_income': -660000000, 'operating_cash_flow': -226000000, 'total_assets': 3767000000, 'total_liabilities': 3223000000}
{'company_name': 'AMD', 'fiscal_year': 2016, 'revenue': 4319000000, 'net_income': -498000000, 'operating_cash_flow': 81000000, 'total_assets': 3321000000, 'total_liabilities': 2777000000}
{'company_name': 'AMD', 'fiscal_year': 2017, 'revenue': 5253000000, 'net_income': -33000000, 'operating_cash_flow': 12000000, 'total_assets': 3540000000, 'total_liabilities': 2929000000}
{'company_name': 'AMD', 'fiscal_year': 2018, 'revenue': 6475000000, 'net_income': 337000000, 'operating_cash_flow': 34000000, 'total_assets': 4556000000, 'total_liabilities': 3290000000}
{'company_name': 'AMD', 'fiscal_year': 2019, 'revenue': 6731000000, 'net_income': 341000000, 'operating_cash_flow': 493000000, 'total_assets': 6028000000, 'total_liabilities': 3201000000}
{'company_name': 'AMD', 'fiscal_year': 2020, 'revenue': 9763

In [24]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [25]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="@Attherate02",
    database="mbis"
)

cursor = conn.cursor()


In [26]:
insert_query = """
INSERT INTO company_financials
(company_name, fiscal_year, revenue, net_income,
 operating_cash_flow, total_assets, total_liabilities)
VALUES (%s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    revenue = VALUES(revenue),
    net_income = VALUES(net_income),
    operating_cash_flow = VALUES(operating_cash_flow),
    total_assets = VALUES(total_assets),
    total_liabilities = VALUES(total_liabilities);
"""


In [27]:
for row in final_data:
    cursor.execute(insert_query, (
        row["company_name"],
        row["fiscal_year"],
        row["revenue"],
        row["net_income"],
        row["operating_cash_flow"],
        row["total_assets"],
        row["total_liabilities"]
    ))

conn.commit()
cursor.close()
conn.close()

In [28]:
import csv

with open("amd_financials.csv", "w", newline="") as f:
    writer = csv.writer(f)
    
    # header (must match MySQL column order)
    writer.writerow([
        "company_name",
        "fiscal_year",
        "revenue",
        "net_income",
        "operating_cash_flow",
        "total_assets",
        "total_liabilities"
    ])
    
    for row in final_data:
        writer.writerow([
            row["company_name"],
            row["fiscal_year"],
            row["revenue"],
            row["net_income"],
            row["operating_cash_flow"],
            row["total_assets"],
            row["total_liabilities"]
        ])


In [25]:
def mysql_safe(value):
    return r"\N" if value is None else value


with open("company_financials.csv", "w", newline="") as f:
    writer = csv.writer(f)

    writer.writerow([
        "company_name",
        "fiscal_year",
        "revenue",
        "net_income",
        "operating_cash_flow",
        "total_assets",
        "total_liabilities"
    ])

    for row in final_data:
        writer.writerow([
            row["company_name"],
            row["fiscal_year"],
            mysql_safe(row["revenue"]),
            mysql_safe(row["net_income"]),
            mysql_safe(row["operating_cash_flow"]),
            mysql_safe(row["total_assets"]),
            mysql_safe(row["total_liabilities"])
        ])
